In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm


from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from nyaggle.feature.category_encoder import TargetEncoder

from nyaggle.experiment import run_experiment

In [2]:
train_log_df = pd.read_csv('../train_log.csv')
test_log_df = pd.read_csv('../test_log.csv')
train_label_df = pd.read_csv('../train_label.csv')
test_session_df = pd.read_csv('../test_session.csv')
yado_df = pd.read_csv('../yado.csv')
# image_embeddings_df = pd.read_parquet('../image_embeddings.parquet')
sample_submission_df = pd.read_csv('../sample_submission.csv')

candidate_train_df = pd.read_csv('../data/lgbBinary_ver3_result_df.csv')
candidate_test_df = pd.read_csv('../data/lgbBinary_ver3_submit_df.csv')

#c = candidate_train_df = pd.read_csv('../data/candidate_ver7_train.csv')

In [3]:
candidate_train_df.drop(['session_id'], axis=1, inplace=True)
candidate_train_df

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,11101,11134,7157,997,5821,12846,12837,8927,4391,1648
1,2570,8253,8140,13717,3184,825,1092,13682,3564,8677
2,9039,13642,918,2957,6722,9854,7796,6676,7509,11031
3,1341,10483,11715,8355,9790,9438,7812,6178,755,1924
4,96,3894,7749,12491,10915,11380,12338,902,1908,5490
...,...,...,...,...,...,...,...,...,...,...
288693,1882,8677,3184,825,13717,13210,1092,13682,2259,3725
288694,2464,7553,4767,8703,8524,8465,1886,10505,1159,5299
288695,12240,6630,7057,4398,9558,2087,11850,7308,3650,10155
288696,10619,570,12500,3238,7551,12829,11316,11091,6933,6595


In [4]:
candidate_test_df.drop(['session_id'], axis=1, inplace=True)
candidate_test_df

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,3560,9830,6766,2040,2680,5466,11496,11561,10965,4948
1,4066,6555,12862,143,11237,1266,7014,5267,11923,7913
2,757,9910,410,2267,11001,6730,7710,6721,10485,10104
3,12341,277,2795,6991,5657,3359,10861,9319,7413,2047
4,12029,5713,3844,9623,12524,6576,5372,13590,3187,6161
...,...,...,...,...,...,...,...,...,...,...
174695,1997,7888,2278,11123,7062,5744,1885,7780,831,8771
174696,1227,6874,4014,5331,3802,12432,13220,2232,13702,9723
174697,12425,12986,10155,7379,12132,8468,6905,3955,13719,8143
174698,6378,11496,3100,10287,2305,2373,8501,2692,12688,5513


#### Trainに正例と負例のフラグを付与

In [5]:
# 学習用データ
# 教師データに正例と負例のフラグを付与
train_target = []

# 各セッションに対して候補の宿をチェック
for index, row in tqdm(train_label_df.iterrows(), total=train_label_df.shape[0]):
    session_id = row['session_id']
    yad_no = row['yad_no']
    
    # candidate_ver7_train.csv の該当行を取得（同じインデックスを仮定）
    candidate_row = candidate_train_df.iloc[index]

    # 50個の購入候補の宿に対してチェック
    for col in candidate_train_df.columns:
        yado_no = candidate_row[col]
        is_correct = (yado_no == yad_no)  # 一致するかどうかのチェック
        train_target.append([session_id, yado_no, is_correct])

# 結果をDataFrameに変換
train_df = pd.DataFrame(train_target, columns=['session_id', 'yado_no', 'target']) #区別するためにあえてyad_noではなくyado_noとしている
train_df['target'] = train_df['target'].astype(int)  # target列をint型に変換

# 最初の数行を表示して確認
print(train_df.head())

print('正例と負例の数を確認')
print(train_df['target'].value_counts())

100%|██████████| 288698/288698 [00:42<00:00, 6850.27it/s]


                         session_id  yado_no  target
0  000007603d533d30453cc45d0f3d119f    11101       0
1  000007603d533d30453cc45d0f3d119f    11134       0
2  000007603d533d30453cc45d0f3d119f     7157       0
3  000007603d533d30453cc45d0f3d119f      997       0
4  000007603d533d30453cc45d0f3d119f     5821       0
正例と負例の数を確認
0    2725823
1     161157
Name: target, dtype: int64


In [6]:
train_df['yado_no'].value_counts()

10095    8853
12524    8468
12350    8443
3338     7998
2445     7811
         ... 
12784       1
542         1
558         1
5591        1
13329       1
Name: yado_no, Length: 11529, dtype: int64

#### 推論用データの作成

In [7]:
# 推論用データにダミーのyado_noを付与
test_session_df['yad_no'] = -1

test_target = []

# 各セッションに対して候補の宿をチェック
for index, row in tqdm(test_session_df.iterrows(), total=test_session_df.shape[0]):
    session_id = row['session_id']
    yad_no = row['yad_no']
    
    # candidate_ver7_test.csv の該当行を取得（同じインデックスを仮定）
    candidate_row = candidate_test_df.iloc[index]

    # 50個の購入候補の宿に対してチェック
    for col in candidate_test_df.columns:
        yado_no = candidate_row[col]
        is_correct = (yado_no == yad_no)  # 一致するかどうかのチェック
        test_target.append([session_id, yado_no, is_correct])

# 結果をDataFrameに変換
test_df = pd.DataFrame(test_target, columns=['session_id', 'yado_no', 'target']) #区別するためにあえてyad_noではなくyado_noとしている
test_df['target'] = test_df['target'].astype(int)  # target列をint型に変換

# 最初の数行を表示して確認
print(test_df.head())

print('正例と負例の数を確認')
print(test_df['target'].value_counts()) # すべて0になるはず

100%|██████████| 174700/174700 [00:25<00:00, 6940.64it/s]


                         session_id  yado_no  target
0  00001149e9c73985425197104712478c     3560       0
1  00001149e9c73985425197104712478c     9830       0
2  00001149e9c73985425197104712478c     6766       0
3  00001149e9c73985425197104712478c     2040       0
4  00001149e9c73985425197104712478c     2680       0
正例と負例の数を確認
0    1747000
Name: target, dtype: int64


#### seq_noを追加

In [8]:
# seq_noをマージする

# train_log の session_id と yad_no の組み合わせに基づいて辞書を作成
seq_no_dict_train = dict(zip(zip(train_log_df['session_id'], train_log_df['yad_no']), train_log_df['seq_no']))
# train_df に seq_no 列を追加（tqdm で進捗表示）
train_df['seq_no'] = [seq_no_dict_train.get((row['session_id'], row['yado_no']), -1) for row in tqdm(train_df.to_dict('records'))]
# 結果の確認
print(train_df.head())


# test_log の session_id と yad_no の組み合わせに基づいて辞書を作成
seq_no_dict_test = dict(zip(zip(test_log_df['session_id'], test_log_df['yad_no']), test_log_df['seq_no']))
# test_df に seq_no 列を追加（tqdm で進捗表示）
test_df['seq_no'] = [seq_no_dict_test.get((row['session_id'], row['yado_no']), -1) for row in tqdm(test_df.to_dict('records'))]
# 結果の確認
print(test_df.head())

100%|██████████| 2886980/2886980 [00:01<00:00, 2357365.93it/s]


                         session_id  yado_no  target  seq_no
0  000007603d533d30453cc45d0f3d119f    11101       0      -1
1  000007603d533d30453cc45d0f3d119f    11134       0      -1
2  000007603d533d30453cc45d0f3d119f     7157       0      -1
3  000007603d533d30453cc45d0f3d119f      997       0      -1
4  000007603d533d30453cc45d0f3d119f     5821       0      -1


100%|██████████| 1747000/1747000 [00:00<00:00, 2506775.98it/s]


                         session_id  yado_no  target  seq_no
0  00001149e9c73985425197104712478c     3560       0       0
1  00001149e9c73985425197104712478c     9830       0      -1
2  00001149e9c73985425197104712478c     6766       0      -1
3  00001149e9c73985425197104712478c     2040       0      -1
4  00001149e9c73985425197104712478c     2680       0      -1


In [9]:
#　※複数のseq_noがある場合は、0も混入する
print(train_df['seq_no'].value_counts())
print(test_df['seq_no'].value_counts())

-1    2777527
 0      88128
 1      16052
 2       4125
 3        837
 4        223
 5         65
 6         18
 7          4
 8          1
Name: seq_no, dtype: int64
-1    1683604
 0      51884
 1       8692
 2       2239
 3        428
 4        124
 5         22
 6          7
Name: seq_no, dtype: int64


#### 候補の宿がsession中に閲覧されていたかどうか

In [10]:
# 'seq_no'が−1でなければ1のフラグを立てる
train_df['seen_yad'] = train_df['seq_no'].apply(lambda x: 1 if x != -1 else 0)
test_df['seen_yad'] = test_df['seq_no'].apply(lambda x: 1 if x != -1 else 0)

In [11]:
print(train_df['seen_yad'].value_counts())  
print(test_df['seen_yad'].value_counts())

0    2777527
1     109453
Name: seen_yad, dtype: int64
0    1683604
1      63396
Name: seen_yad, dtype: int64


#### 各sessionにおける最大seq_no

In [12]:
# 各セッションの seq_no の最大値を計算
max_seq_no_train = train_log_df.groupby('session_id')['seq_no'].max().reset_index()
max_seq_no_train.rename(columns={'seq_no': 'max_seq_no'}, inplace=True)
# teacher_df と max_seq_no のマージ
train_df = train_df.merge(max_seq_no_train, on='session_id', how='left')
# 結果の確認
print(train_df.head())


# 各セッションの seq_no の最大値を計算
max_seq_no_test = test_log_df.groupby('session_id')['seq_no'].max().reset_index()
max_seq_no_test.rename(columns={'seq_no': 'max_seq_no'}, inplace=True)
# teacher_df と max_seq_no のマージ
test_df = test_df.merge(max_seq_no_test, on='session_id', how='left')
# 結果の確認
print(test_df.head())

                         session_id  yado_no  target  seq_no  seen_yad  \
0  000007603d533d30453cc45d0f3d119f    11101       0      -1         0   
1  000007603d533d30453cc45d0f3d119f    11134       0      -1         0   
2  000007603d533d30453cc45d0f3d119f     7157       0      -1         0   
3  000007603d533d30453cc45d0f3d119f      997       0      -1         0   
4  000007603d533d30453cc45d0f3d119f     5821       0      -1         0   

   max_seq_no  
0           0  
1           0  
2           0  
3           0  
4           0  
                         session_id  yado_no  target  seq_no  seen_yad  \
0  00001149e9c73985425197104712478c     3560       0       0         1   
1  00001149e9c73985425197104712478c     9830       0      -1         0   
2  00001149e9c73985425197104712478c     6766       0      -1         0   
3  00001149e9c73985425197104712478c     2040       0      -1         0   
4  00001149e9c73985425197104712478c     2680       0      -1         0   

   max_seq_no 

In [13]:
print(train_df['max_seq_no'].value_counts())
print(test_df['max_seq_no'].value_counts())

0    1853860
1     827930
2     153500
3      40250
4       8330
5       2230
6        650
7        180
8         40
9         10
Name: max_seq_no, dtype: int64
0    1139400
1     494930
2      84590
3      22270
4       4280
5       1240
6        220
7         70
Name: max_seq_no, dtype: int64


#### 差分の考慮：max_seq_noから(-1以外の要素)でseq_noを引く

In [14]:
# 最初に全ての diff_seq_no を -1 に設定
train_df['diff_seq_no'] = -1
test_df['diff_seq_no'] = -1

# seq_no が -1 以外の行にのみ max_seq_no - seq_no の計算を適用
train_df.loc[train_df['seq_no'] != -1, 'diff_seq_no'] = train_df['max_seq_no'] - train_df['seq_no']
test_df.loc[test_df['seq_no'] != -1, 'diff_seq_no'] = test_df['max_seq_no'] - test_df['seq_no']

# 結果の確認
print(train_df.head())
print(test_df.head())

                         session_id  yado_no  target  seq_no  seen_yad  \
0  000007603d533d30453cc45d0f3d119f    11101       0      -1         0   
1  000007603d533d30453cc45d0f3d119f    11134       0      -1         0   
2  000007603d533d30453cc45d0f3d119f     7157       0      -1         0   
3  000007603d533d30453cc45d0f3d119f      997       0      -1         0   
4  000007603d533d30453cc45d0f3d119f     5821       0      -1         0   

   max_seq_no  diff_seq_no  
0           0           -1  
1           0           -1  
2           0           -1  
3           0           -1  
4           0           -1  
                         session_id  yado_no  target  seq_no  seen_yad  \
0  00001149e9c73985425197104712478c     3560       0       0         1   
1  00001149e9c73985425197104712478c     9830       0      -1         0   
2  00001149e9c73985425197104712478c     6766       0      -1         0   
3  00001149e9c73985425197104712478c     2040       0      -1         0   
4  00001149

In [15]:
print(train_df['diff_seq_no'].value_counts())
print(test_df['diff_seq_no'].value_counts())

-1    2777527
 1     103312
 2       5684
 3        407
 4         49
 5          1
Name: diff_seq_no, dtype: int64
-1    1683604
 1      60760
 2       2528
 3        106
 4          2
Name: diff_seq_no, dtype: int64


#### diff_seq_no が奇数かどうかの判定 

In [16]:
# diff_seq_no が奇数かどうかの判定（seq_no が -1 の場合は除外）
train_df['is_odd'] = np.where(train_df['seq_no'] != -1, train_df['diff_seq_no'] % 2 == 1, -1)
test_df['is_odd'] = np.where(test_df['seq_no'] != -1, test_df['diff_seq_no'] % 2 == 1, -1)

# seq_no が -1 以外の場合、Trueを0、Falseを1に変換（すでに -1 の場合は変更しない）
train_df['is_odd'] = np.where(train_df['is_odd'] != -1, np.where(train_df['is_odd'], 0, 1), -1)
test_df['is_odd'] = np.where(test_df['is_odd'] != -1, np.where(test_df['is_odd'], 0, 1), -1)

# 結果の確認
print(train_df.head())
print(test_df.head())

                         session_id  yado_no  target  seq_no  seen_yad  \
0  000007603d533d30453cc45d0f3d119f    11101       0      -1         0   
1  000007603d533d30453cc45d0f3d119f    11134       0      -1         0   
2  000007603d533d30453cc45d0f3d119f     7157       0      -1         0   
3  000007603d533d30453cc45d0f3d119f      997       0      -1         0   
4  000007603d533d30453cc45d0f3d119f     5821       0      -1         0   

   max_seq_no  diff_seq_no  is_odd  
0           0           -1      -1  
1           0           -1      -1  
2           0           -1      -1  
3           0           -1      -1  
4           0           -1      -1  
                         session_id  yado_no  target  seq_no  seen_yad  \
0  00001149e9c73985425197104712478c     3560       0       0         1   
1  00001149e9c73985425197104712478c     9830       0      -1         0   
2  00001149e9c73985425197104712478c     6766       0      -1         0   
3  00001149e9c73985425197104712478c  

In [17]:
print(train_df['is_odd'].value_counts())
print(test_df['is_odd'].value_counts())

-1    2777527
 0     103720
 1       5733
Name: is_odd, dtype: int64
-1    1683604
 0      60866
 1       2530
Name: is_odd, dtype: int64


#### 各sessionにおいて2回以上出現したyad_noがあれば1のフラグを立てる

In [18]:
# 各セッションにおいて2回以上登場する yad_no を特定
yad_no_counts = train_log_df.groupby(['session_id', 'yad_no']).size().reset_index(name='count')
multiple_visits = yad_no_counts[yad_no_counts['count'] > 1]

# (session_id, yad_no) のタプルをキーとして辞書を作成
multiple_visits_dict = {(row['session_id'], row['yad_no']): 1 for _, row in multiple_visits.iterrows()}

# teacher_df に multiple_visits_flag 列を追加（tqdm で進捗表示）
train_df['multiple_visits_flag'] = [multiple_visits_dict.get((row['session_id'], row['yado_no']), 0) 
                                      for row in tqdm(train_df.to_dict('records'))]

# 結果の確認
print(train_df.head())



# 各セッションにおいて2回以上登場する yad_no を特定
yad_no_counts = test_log_df.groupby(['session_id', 'yad_no']).size().reset_index(name='count')
multiple_visits = yad_no_counts[yad_no_counts['count'] > 1]

# (session_id, yad_no) のタプルをキーとして辞書を作成
multiple_visits_dict = {(row['session_id'], row['yad_no']): 1 for _, row in multiple_visits.iterrows()}

# teacher_df に multiple_visits_flag 列を追加（tqdm で進捗表示）
test_df['multiple_visits_flag'] = [multiple_visits_dict.get((row['session_id'], row['yado_no']), 0) 
                                      for row in tqdm(test_df.to_dict('records'))]

# 結果の確認
print(test_df.head())

100%|██████████| 2886980/2886980 [00:00<00:00, 3062141.67it/s]


                         session_id  yado_no  target  seq_no  seen_yad  \
0  000007603d533d30453cc45d0f3d119f    11101       0      -1         0   
1  000007603d533d30453cc45d0f3d119f    11134       0      -1         0   
2  000007603d533d30453cc45d0f3d119f     7157       0      -1         0   
3  000007603d533d30453cc45d0f3d119f      997       0      -1         0   
4  000007603d533d30453cc45d0f3d119f     5821       0      -1         0   

   max_seq_no  diff_seq_no  is_odd  multiple_visits_flag  
0           0           -1      -1                     0  
1           0           -1      -1                     0  
2           0           -1      -1                     0  
3           0           -1      -1                     0  
4           0           -1      -1                     0  


100%|██████████| 1747000/1747000 [00:00<00:00, 3177461.69it/s]


                         session_id  yado_no  target  seq_no  seen_yad  \
0  00001149e9c73985425197104712478c     3560       0       0         1   
1  00001149e9c73985425197104712478c     9830       0      -1         0   
2  00001149e9c73985425197104712478c     6766       0      -1         0   
3  00001149e9c73985425197104712478c     2040       0      -1         0   
4  00001149e9c73985425197104712478c     2680       0      -1         0   

   max_seq_no  diff_seq_no  is_odd  multiple_visits_flag  
0           1            1       0                     0  
1           1           -1      -1                     0  
2           1           -1      -1                     0  
3           1           -1      -1                     0  
4           1           -1      -1                     0  


In [19]:
print(train_df['multiple_visits_flag'].value_counts())
print(test_df['multiple_visits_flag'].value_counts())

0    2882332
1       4648
Name: multiple_visits_flag, dtype: int64
0    1744359
1       2641
Name: multiple_visits_flag, dtype: int64


### yado_dfに関する特徴量エンジニアリング

In [20]:
yado_df

,yad_no,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
0,1,0,129.0,1.0,0,1.0,NaN,NaN,1.0,f0112abf369fb03cdc5f5309300913da,072c85e1653e10c9c7dd065ad007125a,449c52ef581d5f9ef311189469a0520e,677a32689cd1ad74e867f1fbe43a3e1c
1,2,0,23.0,1.0,0,NaN,NaN,NaN,NaN,d86102dd9c232bade9a97dccad40df48,b4d2fb4e51ea7bca80eb1270aa474a54,5c9a8f48e9df0234da012747a02d4b29,4ee16ee838dd2703cc9a1d5a535f0ced
2,3,0,167.0,1.0,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,ab9480fd72a44d51690ab16c4ad4d49c
3,4,0,144.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,52c9ea83f2cfe92be54cb6bc961edf21,1cc3e1838bb0fd0fde0396130b1f82b9
4,5,0,41.0,1.0,1,NaN,NaN,NaN,NaN,43875109d1dab93592812c50d18270a7,75617bb07a2785a948ab1958909211f1,9ea5a911019b66ccd42f556c42a2fe2f,be1b876af18afc4deeb3081591d2a910
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13801,13802,0,10.0,1.0,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,558ac1909f0318b82c621ab250329d6d,80fb3c5ad0c89931d0923e9f80885218,5eb30820716082c720836733d73c605e
13802,13803,0,NaN,NaN,0,1.0,NaN,NaN,1.0,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,e5cfcc0a43c82072aca11628ff0add53,20ad8785a30f125bee5a8a325782ab06
13803,13804,0,80.0,1.0,1,NaN,1.0,NaN,1.0,d86102dd9c232bade9a97dccad40df48,7d76599bd27ff9e7823b2b1323ca763e,c5fe8848b6ab39b040cdb3668aea9433,b3eab50ccf6ffb51c37d36ee384abfbf
13804,13805,0,8.0,1.0,1,NaN,NaN,NaN,1.0,3300cf6f774b7c6a5807110f244cbc21,689cf8289e7ea0b2eef1b017dcdfe8de,8b712435430a6875839a6c3b5a40b008,2b4165444a777465576b25f65697d739


In [21]:
# train_log_df と test_log_df を結合
log_df = pd.concat([train_log_df, test_log_df], axis=0, ignore_index=True)
# yado_df と結合
merged_df = log_df.merge(yado_df, on='yad_no', how='left')


# 各宿の人気度を計算（例：訪問回数で計算）
yad_popularity = merged_df['yad_no'].value_counts().reset_index()
yad_popularity.columns = ['yad_no', 'popularity']

# 全体での人気度ランキング
yad_popularity['overall_rank'] = yad_popularity['popularity'].rank(ascending=False)
# yado_df に人気度をマージ
yado_df = yado_df.merge(yad_popularity, on='yad_no', how='left')
# エリアごとの人気度ランキングを計算
for area in ['wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd']:
    yado_df[f'{area}_rank'] = yado_df.groupby(area)['popularity'].rank(ascending=False, method='min')

# 結果の確認
print(yado_df.head())

   yad_no  yad_type  total_room_cnt  wireless_lan_flg  onsen_flg  kd_stn_5min  \
0       1         0           129.0               1.0          0          1.0   
1       2         0            23.0               1.0          0          NaN   
2       3         0           167.0               1.0          1          1.0   
3       4         0           144.0               1.0          0          1.0   
4       5         0            41.0               1.0          1          NaN   

   kd_bch_5min  kd_slp_5min  kd_conv_walk_5min  \
0          NaN          NaN                1.0   
1          NaN          NaN                NaN   
2          NaN          NaN                1.0   
3          NaN          NaN                1.0   
4          NaN          NaN                NaN   

                             wid_cd                            ken_cd  \
0  f0112abf369fb03cdc5f5309300913da  072c85e1653e10c9c7dd065ad007125a   
1  d86102dd9c232bade9a97dccad40df48  b4d2fb4e51ea7bca80eb1270aa474

In [22]:
#　train_dfとtest_dfにyado_dfをマージ
train_df = pd.merge(train_df, yado_df.rename(columns={'yad_no': 'yado_no'}), on='yado_no', how='left')
test_df = pd.merge(test_df, yado_df.rename(columns={'yad_no': 'yado_no'}), on='yado_no', how='left')

In [23]:
train_df

,session_id,yado_no,target,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,...,wid_cd,ken_cd,lrg_cd,sml_cd,popularity,overall_rank,wid_cd_rank,ken_cd_rank,lrg_cd_rank,sml_cd_rank
0,000007603d533d30453cc45d0f3d119f,11101,0,-1,0,0,-1,-1,0,0,...,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,196.0,440.5,52.0,32.0,1.0,1.0
1,000007603d533d30453cc45d0f3d119f,11134,0,-1,0,0,-1,-1,0,0,...,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,188.0,486.0,62.0,35.0,2.0,2.0
2,000007603d533d30453cc45d0f3d119f,7157,0,-1,0,0,-1,-1,0,0,...,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,86.0,2418.5,280.0,107.0,5.0,5.0
3,000007603d533d30453cc45d0f3d119f,997,0,-1,0,0,-1,-1,0,0,...,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,73.0,2972.0,366.0,134.0,6.0,6.0
4,000007603d533d30453cc45d0f3d119f,5821,0,-1,0,0,-1,-1,0,0,...,dc414a17890cfc17d011d5038b88ca93,d78f53d0856617bc782f02c3280dfef2,4fd631b15116098340cdb099c86a5a40,4044dac1931ddaa5a967e09506d76343,69.0,3182.0,390.0,144.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886975,fffffa7baf370083ebcdd98f26a7e31a,10439,0,-1,0,1,-1,-1,0,0,...,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,ab9480fd72a44d51690ab16c4ad4d49c,63.0,3503.5,799.0,114.0,52.0,7.0
2886976,fffffa7baf370083ebcdd98f26a7e31a,1372,0,-1,0,1,-1,-1,0,0,...,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,ab9480fd72a44d51690ab16c4ad4d49c,119.0,1434.5,372.0,64.0,43.0,4.0
2886977,fffffa7baf370083ebcdd98f26a7e31a,3848,0,-1,0,1,-1,-1,0,0,...,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,1d9f09b9e2bd43cebc9885a46388739a,1018.0,11.0,8.0,8.0,8.0,7.0
2886978,fffffa7baf370083ebcdd98f26a7e31a,846,0,-1,0,1,-1,-1,0,0,...,46e33861f921c3e38b81998fbf283f01,572d60f0f5212aacda515ebf81fb0a3a,8a623b960557e87bd1f4edf71b6255be,ab9480fd72a44d51690ab16c4ad4d49c,145.0,915.0,247.0,50.0,38.0,2.0


#### 各種Encoding

In [24]:
# カテゴリ変数と数値変数の明確化
cat_cols = ['wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd']

# 欠損値を-1で埋める
train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)

In [25]:
# Label Encoding

# testにしか存在しないものはNullにするので、trainのみをencoderにする
for col in cat_cols:
    encoder = {c: i for i, c in enumerate(train_df[col].unique())}
    train_df[f'label_{col}'] = train_df[col].map(encoder)
    test_df[f'label_{col}'] = test_df[col].map(encoder)

In [26]:
# Count Encoding（trainとtestをマージした方が良い？）←　した
# 増やした方が良い？

cat_cols = ['wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd'] + ['yado_no']

# testにしか存在しないものはNullにするので、trainのみをencoderにする
for col in cat_cols:
    encoder = train_df[col].value_counts()
    train_df[f'count_{col}'] = train_df[col].map(encoder)
    test_df[f'count_{col}'] = test_df[col].map(encoder)

In [27]:
# Target encoding
# 元の列順が保持されないから

cat_cols = ['wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd'] + ['yado_no']

# GroupKFold の設定  StratifiedGrouoKFoldでもいいかもしれない
gkf = GroupKFold(n_splits=5)

# fold 列を初期化
train_df['fold'] = -1

# 各 Fold に対してインデックスを割り当て
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups=train_df['session_id'])):
    # バリデーションセットのインデックスに Fold 番号を割り当て
    train_df.loc[val_idx, 'fold'] = fold
# 結果の確認
print(train_df.head())



# TargetEncodingが完了した検証用データセットを格納するリスト
encoded_dfs = []

# すべての分割についてのループ
for fold in range(5):

  # 学習用と検証用データセットに分割する
  df_train = train_df[train_df.fold != fold].reset_index(drop=True)
  df_valid = train_df[train_df.fold == fold].reset_index(drop=True)

  # すべてのカテゴリについてのループ
  for column in cat_cols:
    # カテゴリごとの目的変数の平均についての辞書を作成
    mapping_dict = dict(
        df_train.groupby(column)['target'].mean()
    )
    # 元の列名の末尾に'enc'を加えた名前で、新しい列を作成
    df_valid.loc[
        :, 'TE_' + column
    ] = df_valid[column].map(mapping_dict)

  # リストに格納
  encoded_dfs.append(df_valid)

# 結合したデータセットを返す
encoded_df = pd.concat(encoded_dfs, axis=0)


                         session_id  yado_no  target  seq_no  seen_yad  \
0  000007603d533d30453cc45d0f3d119f    11101       0      -1         0   
1  000007603d533d30453cc45d0f3d119f    11134       0      -1         0   
2  000007603d533d30453cc45d0f3d119f     7157       0      -1         0   
3  000007603d533d30453cc45d0f3d119f      997       0      -1         0   
4  000007603d533d30453cc45d0f3d119f     5821       0      -1         0   

   max_seq_no  diff_seq_no  is_odd  multiple_visits_flag  yad_type  ...  \
0           0           -1      -1                     0         0  ...   
1           0           -1      -1                     0         0  ...   
2           0           -1      -1                     0         0  ...   
3           0           -1      -1                     0         0  ...   
4           0           -1      -1                     0         0  ...   

   label_wid_cd  label_ken_cd  label_lrg_cd  label_sml_cd  count_wid_cd  \
0             0             0

In [28]:
train_df

,session_id,yado_no,target,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,...,label_wid_cd,label_ken_cd,label_lrg_cd,label_sml_cd,count_wid_cd,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,fold
0,000007603d533d30453cc45d0f3d119f,11101,0,-1,0,0,-1,-1,0,0,...,0,0,0,0,329243,138825,6712,6073,558,2
1,000007603d533d30453cc45d0f3d119f,11134,0,-1,0,0,-1,-1,0,0,...,0,0,0,0,329243,138825,6712,6073,568,2
2,000007603d533d30453cc45d0f3d119f,7157,0,-1,0,0,-1,-1,0,0,...,0,0,0,0,329243,138825,6712,6073,579,2
3,000007603d533d30453cc45d0f3d119f,997,0,-1,0,0,-1,-1,0,0,...,0,0,0,0,329243,138825,6712,6073,585,2
4,000007603d533d30453cc45d0f3d119f,5821,0,-1,0,0,-1,-1,0,0,...,0,0,0,0,329243,138825,6712,6073,557,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886975,fffffa7baf370083ebcdd98f26a7e31a,10439,0,-1,0,1,-1,-1,0,0,...,2,2,12,52,695344,128535,81656,7233,579,0
2886976,fffffa7baf370083ebcdd98f26a7e31a,1372,0,-1,0,1,-1,-1,0,0,...,2,2,12,52,695344,128535,81656,7233,694,0
2886977,fffffa7baf370083ebcdd98f26a7e31a,3848,0,-1,0,1,-1,-1,0,0,...,2,2,12,16,695344,128535,81656,61805,5113,0
2886978,fffffa7baf370083ebcdd98f26a7e31a,846,0,-1,0,1,-1,-1,0,0,...,2,2,12,52,695344,128535,81656,7233,372,0


In [29]:
train_df = encoded_df

In [30]:
# Testデータに対しても同様にTargetEncodingを行う
cat_cols = ['wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd'] + ['yado_no']

# すべてのカテゴリについてのループ
# train_df全体で計算する
for column in cat_cols:
    # カテゴリごとの目的変数の平均についての辞書を作成
    mapping_dict = dict(
        train_df.groupby(column)['target'].mean()
    )
    
    # 元の列名の末尾に'enc'を加えた名前で、新しい列を作成
    test_df.loc[
        :, 'TE_' + column
    ] = test_df[column].map(mapping_dict)

In [31]:
test_df

,session_id,yado_no,target,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,...,count_wid_cd,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,TE_wid_cd,TE_ken_cd,TE_lrg_cd,TE_sml_cd,TE_yado_no
0,00001149e9c73985425197104712478c,3560,0,0,1,1,1,0,0,0,...,695344,387623,25665,10349,836.0,0.052088,0.045766,0.050653,0.049377,0.040670
1,00001149e9c73985425197104712478c,9830,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,14422,2592.0,0.052088,0.045766,0.050653,0.050894,0.060957
2,00001149e9c73985425197104712478c,6766,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,14422,1766.0,0.052088,0.045766,0.050653,0.050894,0.056625
3,00001149e9c73985425197104712478c,2040,0,-1,0,1,-1,-1,0,0,...,695344,387623,18239,5043,1038.0,0.052088,0.045766,0.048906,0.054333,0.048170
4,00001149e9c73985425197104712478c,2680,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,10349,1117.0,0.052088,0.045766,0.050653,0.049377,0.053715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746995,ffffe984aafd6127ce8e43e3ca40c79d,5623,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,1287.0,0.052088,0.045766,0.041930,0.043284,0.034188
1746996,ffffe984aafd6127ce8e43e3ca40c79d,3781,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,15222,1867.0,0.052088,0.045766,0.041930,0.038497,0.028388
1746997,ffffe984aafd6127ce8e43e3ca40c79d,13593,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,1214.0,0.052088,0.045766,0.041930,0.043284,0.026359
1746998,ffffe984aafd6127ce8e43e3ca40c79d,12767,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,924.0,0.052088,0.045766,0.041930,0.043284,0.027056


In [32]:
all_features = train_df.columns.to_list()
features= [x for x in all_features if x not in ('session_id', 'fold', 'target', 'wid_cd', 'ken_cd', 'lrg_cd', 'sml_cd')]

In [33]:
train_df[features]

,yado_no,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,total_room_cnt,wireless_lan_flg,...,count_wid_cd,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,TE_wid_cd,TE_ken_cd,TE_lrg_cd,TE_sml_cd,TE_yado_no
0,96,0,1,1,1,0,0,0,228.0,1.0,...,354410,38488,14600,13807,136,0.051931,0.057987,0.058557,0.058137,0.079646
1,3894,-1,0,1,-1,-1,0,0,205.0,1.0,...,354410,38488,14600,13807,1320,0.051931,0.057987,0.058557,0.058137,0.063256
2,7749,-1,0,1,-1,-1,0,0,76.0,1.0,...,354410,38488,14600,13807,1333,0.051931,0.057987,0.058557,0.058137,0.058226
3,12491,-1,0,1,-1,-1,0,0,144.0,1.0,...,354410,38488,14600,13807,1327,0.051931,0.057987,0.058557,0.058137,0.046468
4,10915,-1,0,1,-1,-1,0,0,100.0,1.0,...,354410,38488,14600,13807,1284,0.051931,0.057987,0.058557,0.058137,0.044188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577385,7407,-1,0,1,-1,-1,0,0,34.0,1.0,...,92119,12163,10880,8713,829,0.059521,0.068107,0.063658,0.065081,0.058471
577386,5076,-1,0,1,-1,-1,0,0,207.0,1.0,...,92119,12163,10880,8713,937,0.059521,0.068107,0.063658,0.065081,0.052562
577387,1502,-1,0,1,-1,-1,0,0,44.0,1.0,...,92119,12163,10880,8713,646,0.059521,0.068107,0.063658,0.065081,0.045714
577388,3271,-1,0,1,-1,-1,0,0,138.0,1.0,...,92119,12163,10880,8713,317,0.059521,0.068107,0.063658,0.065081,0.047872


In [34]:
test_df[features]

,yado_no,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,total_room_cnt,wireless_lan_flg,...,count_wid_cd,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,TE_wid_cd,TE_ken_cd,TE_lrg_cd,TE_sml_cd,TE_yado_no
0,3560,0,1,1,1,0,0,0,205.0,1.0,...,695344,387623,25665,10349,836.0,0.052088,0.045766,0.050653,0.049377,0.040670
1,9830,-1,0,1,-1,-1,0,0,384.0,1.0,...,695344,387623,25665,14422,2592.0,0.052088,0.045766,0.050653,0.050894,0.060957
2,6766,-1,0,1,-1,-1,0,0,89.0,1.0,...,695344,387623,25665,14422,1766.0,0.052088,0.045766,0.050653,0.050894,0.056625
3,2040,-1,0,1,-1,-1,0,0,193.0,1.0,...,695344,387623,18239,5043,1038.0,0.052088,0.045766,0.048906,0.054333,0.048170
4,2680,-1,0,1,-1,-1,0,0,150.0,1.0,...,695344,387623,25665,10349,1117.0,0.052088,0.045766,0.050653,0.049377,0.053715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746995,5623,-1,0,0,-1,-1,0,0,178.0,1.0,...,695344,387623,37825,12730,1287.0,0.052088,0.045766,0.041930,0.043284,0.034188
1746996,3781,-1,0,0,-1,-1,0,0,245.0,1.0,...,695344,387623,37825,15222,1867.0,0.052088,0.045766,0.041930,0.038497,0.028388
1746997,13593,-1,0,0,-1,-1,0,0,112.0,1.0,...,695344,387623,37825,12730,1214.0,0.052088,0.045766,0.041930,0.043284,0.026359
1746998,12767,-1,0,0,-1,-1,0,0,270.0,1.0,...,695344,387623,37825,12730,924.0,0.052088,0.045766,0.041930,0.043284,0.027056


In [35]:
train_df.columns.to_list()

['session_id',
 'yado_no',
 'target',
 'seq_no',
 'seen_yad',
 'max_seq_no',
 'diff_seq_no',
 'is_odd',
 'multiple_visits_flag',
 'yad_type',
 'total_room_cnt',
 'wireless_lan_flg',
 'onsen_flg',
 'kd_stn_5min',
 'kd_bch_5min',
 'kd_slp_5min',
 'kd_conv_walk_5min',
 'wid_cd',
 'ken_cd',
 'lrg_cd',
 'sml_cd',
 'popularity',
 'overall_rank',
 'wid_cd_rank',
 'ken_cd_rank',
 'lrg_cd_rank',
 'sml_cd_rank',
 'label_wid_cd',
 'label_ken_cd',
 'label_lrg_cd',
 'label_sml_cd',
 'count_wid_cd',
 'count_ken_cd',
 'count_lrg_cd',
 'count_sml_cd',
 'count_yado_no',
 'fold',
 'TE_wid_cd',
 'TE_ken_cd',
 'TE_lrg_cd',
 'TE_sml_cd',
 'TE_yado_no']

In [36]:
import gc
gc.collect()

189

In [37]:
test_df

,session_id,yado_no,target,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,...,count_wid_cd,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,TE_wid_cd,TE_ken_cd,TE_lrg_cd,TE_sml_cd,TE_yado_no
0,00001149e9c73985425197104712478c,3560,0,0,1,1,1,0,0,0,...,695344,387623,25665,10349,836.0,0.052088,0.045766,0.050653,0.049377,0.040670
1,00001149e9c73985425197104712478c,9830,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,14422,2592.0,0.052088,0.045766,0.050653,0.050894,0.060957
2,00001149e9c73985425197104712478c,6766,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,14422,1766.0,0.052088,0.045766,0.050653,0.050894,0.056625
3,00001149e9c73985425197104712478c,2040,0,-1,0,1,-1,-1,0,0,...,695344,387623,18239,5043,1038.0,0.052088,0.045766,0.048906,0.054333,0.048170
4,00001149e9c73985425197104712478c,2680,0,-1,0,1,-1,-1,0,0,...,695344,387623,25665,10349,1117.0,0.052088,0.045766,0.050653,0.049377,0.053715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746995,ffffe984aafd6127ce8e43e3ca40c79d,5623,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,1287.0,0.052088,0.045766,0.041930,0.043284,0.034188
1746996,ffffe984aafd6127ce8e43e3ca40c79d,3781,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,15222,1867.0,0.052088,0.045766,0.041930,0.038497,0.028388
1746997,ffffe984aafd6127ce8e43e3ca40c79d,13593,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,1214.0,0.052088,0.045766,0.041930,0.043284,0.026359
1746998,ffffe984aafd6127ce8e43e3ca40c79d,12767,0,-1,0,0,-1,-1,0,0,...,695344,387623,37825,12730,924.0,0.052088,0.045766,0.041930,0.043284,0.027056


In [38]:
train_df

,session_id,yado_no,target,seq_no,seen_yad,max_seq_no,diff_seq_no,is_odd,multiple_visits_flag,yad_type,...,count_ken_cd,count_lrg_cd,count_sml_cd,count_yado_no,fold,TE_wid_cd,TE_ken_cd,TE_lrg_cd,TE_sml_cd,TE_yado_no
0,000104bdffaaad1a1e0a9ebacf585f33,96,1,0,1,1,1,0,0,0,...,38488,14600,13807,136,0,0.051931,0.057987,0.058557,0.058137,0.079646
1,000104bdffaaad1a1e0a9ebacf585f33,3894,0,-1,0,1,-1,-1,0,0,...,38488,14600,13807,1320,0,0.051931,0.057987,0.058557,0.058137,0.063256
2,000104bdffaaad1a1e0a9ebacf585f33,7749,0,-1,0,1,-1,-1,0,0,...,38488,14600,13807,1333,0,0.051931,0.057987,0.058557,0.058137,0.058226
3,000104bdffaaad1a1e0a9ebacf585f33,12491,0,-1,0,1,-1,-1,0,0,...,38488,14600,13807,1327,0,0.051931,0.057987,0.058557,0.058137,0.046468
4,000104bdffaaad1a1e0a9ebacf585f33,10915,0,-1,0,1,-1,-1,0,0,...,38488,14600,13807,1284,0,0.051931,0.057987,0.058557,0.058137,0.044188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577385,fffe8c99c5b332190c3d4a2d6e7c5073,7407,0,-1,0,1,-1,-1,0,0,...,12163,10880,8713,829,4,0.059521,0.068107,0.063658,0.065081,0.058471
577386,fffe8c99c5b332190c3d4a2d6e7c5073,5076,0,-1,0,1,-1,-1,0,0,...,12163,10880,8713,937,4,0.059521,0.068107,0.063658,0.065081,0.052562
577387,fffe8c99c5b332190c3d4a2d6e7c5073,1502,0,-1,0,1,-1,-1,0,0,...,12163,10880,8713,646,4,0.059521,0.068107,0.063658,0.065081,0.045714
577388,fffe8c99c5b332190c3d4a2d6e7c5073,3271,0,-1,0,1,-1,-1,0,0,...,12163,10880,8713,317,4,0.059521,0.068107,0.063658,0.065081,0.047872


In [39]:
# train_df.to_csv('../feature_engineering_v3_train_df.csv', index=False)
# test_df.to_csv('../feature_engineering_v3_test_df.csv', index=False)

In [40]:
train_df.to_parquet('../data/ReRank2nd_ver1_train_df.parquet', index=False)
test_df.to_parquet('../data/ReRank2nd_ver1_test_df.parquet', index=False)